In [1]:

from happytransformer import HappyTextToText

import errant,csv
ANNOTATOR = errant.load('en')
happy_tt = HappyTextToText("T5", "t5-base")


/home/ryanpapetti/Desktop/Programming/Professional/Feedback Fruits/Interview Assignment/venv/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
06/12/2022 01:33:22 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [2]:
import gc
# del variables
gc.collect()
import torch
torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |     850 MB |     850 MB |     850 MB |       0 B  |
|       from large pool |     850 MB |     850 MB |     850 MB |       0 B  |
|       from small pool |       0 MB |       0 MB |       0 MB |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |     850 MB |     850 MB |     850 MB |       0 B  |
|       from large pool |     850 MB |     850 MB |     850 MB |

In [3]:
replacements = [
  (" .", "."), 
  (" ,", ","),
  (" '", "'"),
  (" ?", "?"),
  (" !", "!"),
  (" :", "!"),
  (" ;", "!"),
  (" n't", "n't"),
  (" v", "n't"),
  (" 've", "'ve"),
  ("2 0 0 6", "2006"),
  ("5 5", "55"),
  ("4 0 0", "400"),
  ("1 7-5 0", "1750"),
  ("2 0 %", "20%"),
  ("5 0", "50"),
  ("1 2", "12"),
  ("1 0", "10"),
  ('" ballast water', '"ballast water') #specific example
]

def remove_excess_spaces(text):
  for rep in replacements:
    text = text.replace(rep[0], rep[1])

  return text

In [4]:
def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                # a few of the cases are blank strings. So we'll skip them
                if input_text and correction:
                    writter.writerow([remove_excess_spaces(input_text), remove_excess_spaces(correction)])

In [6]:
from datasets import load_dataset
train_dataset = load_dataset("jfleg", split='validation[:]')
eval_dataset = load_dataset("jfleg", split='test[:]')
generate_csv("data/jfleg_train.csv", train_dataset)
generate_csv("data/jfleg_eval.csv", eval_dataset)

In [7]:
before_result = happy_tt.eval("data/jfleg_eval.csv")
print("Before loss: ", before_result.loss)

06/12/2022 01:37:08 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/ryanpapetti/.cache/huggingface/datasets/csv/default-6bfa9a9f53efaa7a/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 2988
  Batch size = 1


  0%|          | 0/2988 [00:00<?, ?it/s]

Before loss:  1.4225281476974487


In [22]:
from happytransformer import TTTrainArgs
args = TTTrainArgs(batch_size=1, num_train_epochs=4)
happy_tt.train("data/jfleg_train.csv", args=args)

06/12/2022 02:02:32 - INFO - happytransformer.happy_transformer -   Preprocessing training data...


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

06/12/2022 02:02:33 - INFO - happytransformer.happy_transformer -   Training...
PyTorch: setting up devices
***** Running training *****
  Num examples = 3016
  Num Epochs = 4
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 12064


  0%|          | 0/12064 [00:00<?, ?it/s]

{'loss': 0.4025, 'learning_rate': 4.792771883289125e-05, 'epoch': 0.17}
{'loss': 0.4068, 'learning_rate': 4.5855437665782495e-05, 'epoch': 0.33}
{'loss': 0.3975, 'learning_rate': 4.3783156498673745e-05, 'epoch': 0.5}
{'loss': 0.4303, 'learning_rate': 4.171087533156499e-05, 'epoch': 0.66}
{'loss': 0.4079, 'learning_rate': 3.963859416445623e-05, 'epoch': 0.83}
{'loss': 0.4135, 'learning_rate': 3.756631299734748e-05, 'epoch': 0.99}
{'loss': 0.2932, 'learning_rate': 3.549403183023873e-05, 'epoch': 1.16}
{'loss': 0.3331, 'learning_rate': 3.3421750663129974e-05, 'epoch': 1.33}
{'loss': 0.3546, 'learning_rate': 3.1349469496021224e-05, 'epoch': 1.49}
{'loss': 0.3377, 'learning_rate': 2.927718832891247e-05, 'epoch': 1.66}
{'loss': 0.3597, 'learning_rate': 2.7204907161803717e-05, 'epoch': 1.82}
{'loss': 0.3533, 'learning_rate': 2.5132625994694963e-05, 'epoch': 1.99}
{'loss': 0.2808, 'learning_rate': 2.306034482758621e-05, 'epoch': 2.16}
{'loss': 0.275, 'learning_rate': 2.0988063660477456e-05, 'e



Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 1705.5023, 'train_samples_per_second': 7.074, 'train_steps_per_second': 7.074, 'train_loss': 0.32182312343734015, 'epoch': 4.0}


In [23]:
after_result = happy_tt.eval("data/jfleg_eval.csv")
print("After loss:", after_result.loss)

06/12/2022 02:30:59 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 2988
  Batch size = 1


  0%|          | 0/2988 [00:00<?, ?it/s]

After loss: 0.5540881752967834


In [24]:
# device = "cuda:0"
# correction_model_tag = "vennify/t5-base-grammar-correction"
# correction_tokenizer = AutoTokenizer.from_pretrained(correction_model_tag)
# correction_model = AutoModelForSeq2SeqLM.from_pretrained(correction_model_tag).to(device)

In [25]:
def correct(input_sentence, max_candidates=1):

    correction_prefix = "gec: "
    input_sentence = correction_prefix + input_sentence
    input_ids = happy_tt.tokenizer.encode(input_sentence, return_tensors='pt')
    input_ids = input_ids.to(device)

    preds = happy_tt.model.generate(
        input_ids,
        do_sample=True, 
        max_length=128, 
        top_k=50, 
        top_p=1.5, 
        early_stopping=True,
        num_return_sequences=max_candidates)

    corrected = set()
    for pred in preds:  
        corrected.add((happy_tt.tokenizer.decode(pred, skip_special_tokens=True).strip()))
    return corrected


In [32]:
def _get_edits(orig, cor):
    orig = ANNOTATOR.parse(orig)
    cor = ANNOTATOR.parse(cor)
    alignment = ANNOTATOR.align(orig, cor)
    edits = ANNOTATOR.merge(alignment)

    if len(edits) == 0:  
        return []

    edit_annotations = []
    for e in edits:
        e = ANNOTATOR.classify(e)
        edit_annotations.append((e.type[2:], e.o_str, e.o_start, e.o_end,  e.c_str, e.c_start, e.c_end))
            
    if len(edit_annotations) > 0:
        return edit_annotations
    else:    
        return []

def get_edits(orig, cor):
    return _get_edits(orig, cor)

In [35]:
device = "cuda:0"

influent_sentences = [
    "He are moving here.",
    "the collection of letters was original used by the ancient Romans",
    "We enjoys horror movies",
    "Anna and Mike is going skiing without us",
    "I will eat fish for dinner and drank milk before sleep",
    "what be the reason for everyone leave the comapny"
]


corrections = {}

for influent_sentence in influent_sentences:
    corrected_sentences = correct(influent_sentence, max_candidates=1)
    print("[Input] ", influent_sentence)
    for corrected_sentence in corrected_sentences:
      print("[OUTPUT]", corrected_sentence)
      print("[Edits] ", get_edits(influent_sentence, corrected_sentence))
      corrections[influent_sentence] = (corrected_sentence,get_edits(influent_sentence, corrected_sentence))
    print("-" *100)

[Input]  He are moving here.
[OUTPUT] He are moving here.
[Edits]  []
----------------------------------------------------------------------------------------------------
[Input]  the collection of letters was original used by the ancient Romans
[OUTPUT] The collection of letters was original and used by the ancient Romans.
[Edits]  [('ORTH', 'the', 0, 1, 'The', 0, 1), ('CONJ', '', 6, 6, 'and', 6, 7), ('NOUN', 'Romans', 10, 11, 'Romans.', 11, 12)]
----------------------------------------------------------------------------------------------------
[Input]  We enjoys horror movies
[OUTPUT] We enjoy horror movies.
[Edits]  [('VERB:SVA', 'enjoys', 1, 2, 'enjoy', 1, 2), ('NOUN', 'movies', 3, 4, 'movies.', 3, 4)]
----------------------------------------------------------------------------------------------------
[Input]  Anna and Mike is going skiing without us
[OUTPUT] Anna and Mike are going skiing without us.
[Edits]  [('VERB:SVA', 'is', 3, 4, 'are', 3, 4), ('OTHER', 'us', 7, 8, 'us.', 7,

In [38]:
for original_sentence, proposed_edits in corrections.items():
    new_sentence, edits = proposed_edits
    print(f"In order to improve your original sentence:\n\n {original_sentence}\n\ntry the proposed changes:\n")
    for ind,edit in enumerate(edits):
        print(f"{ind+1}: Change '{edit[1]}' to '{edit[4]}' ({edit[0]})\n\n")
    print(f"After changes, the new sentence could read: {new_sentence}\n\n")

In order to improve your original sentence:

 He are moving here.

try the proposed changes:

After changes, the new sentence could read: He are moving here.


In order to improve your original sentence:

 the collection of letters was original used by the ancient Romans

try the proposed changes:

1: Change 'the' to 'The' (ORTH)


2: Change '' to 'and' (CONJ)


3: Change 'Romans' to 'Romans.' (NOUN)


After changes, the new sentence could read: The collection of letters was original and used by the ancient Romans.


In order to improve your original sentence:

 We enjoys horror movies

try the proposed changes:

1: Change 'enjoys' to 'enjoy' (VERB:SVA)


2: Change 'movies' to 'movies.' (NOUN)


After changes, the new sentence could read: We enjoy horror movies.


In order to improve your original sentence:

 Anna and Mike is going skiing without us

try the proposed changes:

1: Change 'is' to 'are' (VERB:SVA)


2: Change 'us' to 'us.' (OTHER)


After changes, the new sentence could r

In [29]:
happy_tt.save("model_dir")

Configuration saved in model_dir/config.json
Model weights saved in model_dir/pytorch_model.bin
tokenizer config file saved in model_dir/tokenizer_config.json
Special tokens file saved in model_dir/special_tokens_map.json
Copy vocab file to model_dir/spiece.model


In [40]:
import json
json.dumps(corrections)

'{"He are moving here.": ["He are moving here.", []], "the collection of letters was original used by the ancient Romans": ["The collection of letters was original and used by the ancient Romans.", [["ORTH", "the", 0, 1, "The", 0, 1], ["CONJ", "", 6, 6, "and", 6, 7], ["NOUN", "Romans", 10, 11, "Romans.", 11, 12]]], "We enjoys horror movies": ["We enjoy horror movies.", [["VERB:SVA", "enjoys", 1, 2, "enjoy", 1, 2], ["NOUN", "movies", 3, 4, "movies.", 3, 4]]], "Anna and Mike is going skiing without us": ["Anna and Mike are going skiing without us.", [["VERB:SVA", "is", 3, 4, "are", 3, 4], ["OTHER", "us", 7, 8, "us.", 7, 8]]], "I will eat fish for dinner and drank milk before sleep": ["I will eat fish for dinner and drink milk before sleeping.", [["VERB:TENSE", "drank", 7, 8, "drink", 7, 8], ["NOUN", "sleep", 10, 11, "sleeping.", 10, 11]]], "what be the reason for everyone leave the comapny": ["What is the reason for everybody to leave the colony?", [["ORTH", "what", 0, 1, "What", 0, 1], 